# Re-run hate terms
After we shared our findings with Google and got their statement, we re-collected the data and checked what changed.

This notebook processes the API responses for the keywords and compares the new API statuses with the original API statuses.

In [1]:
import os
import json
import glob

from tqdm import tqdm
import pandas as pd
from utils import value_counts, determine_status, process_api_response

In [2]:
# input
data_in = '../data/z_data_rerun/input/placements_api/'
fn_hate_og = '../data/output/placements_api_keyword_status/hate.csv'
fn_social_justice_og = '../data/output/placements_api_keyword_status/social_justice.csv'
data_dir_religions = '../data/z_data_rerun/input/placements_api/religions/'

# outputs
data_dir = '../data/z_data_rerun/output/placements_api_keyword_status/'
fn_hate = os.path.join(data_dir, "hate.csv")
fn_social_justice = os.path.join(data_dir, "social_justice.csv")
fn_policy = os.path.join(data_dir, "policy.csv")
fn_basewords = os.path.join(data_dir, "basewords.csv")
fn_adhoc = os.path.join(data_dir, "adhoc.csv")
os.makedirs(data_dir, exist_ok=True)

fn_hate_table4 = '../data/output/tables/hate/table4.csv'
fn_socjust_table4 = '../data/output/tables/socialjustice/table4.csv'

In [3]:
save_cols = [
    'search_term', 
    'status',
    'status_no_spaces',
    'n_youtube_videos',
    'n_youtube_channels',
    'n_youtube_videos_no_spaces',
    'n_youtube_channels_no_spaces',
]

display_cols = ['search_term', 'status', 'status_no_spaces']

In [4]:
files = glob.glob(data_in + '*/*.json')
len(files)

448

## Checking the API responses
Use the `utils` functions `process_api_response` and `determine_status` from the original API responses in `1-data-preprocessing.ipynb`.

In [5]:
dataset = []
for fn in tqdm(files):
    record = process_api_response(fn)
    dataset.append(record)
    
df = pd.DataFrame(dataset)

100%|██████████| 448/448 [00:00<00:00, 24015.24it/s]


In [6]:
df['status'] = df.apply(determine_status, axis=1)

In [7]:
merge_cols = [
    'search_term', 
    'is_blocked', 
    'status', 
    'n_youtube_videos', 
    'n_youtube_channels'
]
blocked = df[df.fn.str.contains('/blocked/')]
df = df.merge(blocked[merge_cols], 
              on=['search_term'], how='left', 
              suffixes=('', '_no_spaces'))

In [8]:
df.sort_values(by=['n_youtube_channels', 
                   'is_blocked_no_spaces',
                   'search_term'], 
               ascending=False, 
               inplace=True)

In [9]:
hate = df[df.fn.str.contains('/hate/')]
social_justice = df[df.fn.str.contains('/social_justice/')]
policy = df[df.fn.str.contains('/policy/')]
word = df[df.fn.str.contains('/blocked_basewords/')]
adhoc = df[(df.fn.str.contains('adhoc/'))]

In [10]:
# save the status of terms from each keyword list
hate[save_cols].to_csv(fn_hate, index=False)
social_justice[save_cols].to_csv(fn_social_justice, index=False)
policy[save_cols].to_csv(fn_policy, index=False)
word[save_cols].to_csv(fn_basewords, index=False)
adhoc[save_cols].to_csv(fn_adhoc, index=False)

## Analyze Hate terms

In [11]:
hate_og = pd.read_csv(fn_hate_og)

In [12]:
# before
value_counts(hate_og, col='status')

,count,percentage
Full,58,0.674419
Blocked,28,0.325581


Half of the terms are now blocked with `Empty` responses. Note this `Empty` response is a totally unique use from the original data collection. This prevents any further studies from differentiating between a blocked search and an obscure search.

In [13]:
# after
value_counts(hate, col='status')

,count,percentage
Empty,44,0.511628
Blocked,28,0.325581
Full,14,0.162791


In [14]:
# compare the original responses to the new responses
hate_delta = hate_og[display_cols].merge(
    hate[display_cols],
    on='search_term', 
    suffixes=['_before', '_after']
)

In [15]:
# hate terms that are now blocked with an "empty" response.
hate_delta[
    hate_delta.status_after != hate_delta.status_before
][['search_term','status_before', 'status_after']].set_index('search_term')

,status_before,status_after
search_term,,
14 words,Full,Empty
2083: a european declaration of independence,Full,Empty
alt-lite,Full,Empty
alt-right,Full,Empty
american front,Full,Empty
american identity movement,Full,Empty
american vanguard,Full,Empty
blood and soil,Full,Empty
civilization jihad,Full,Empty


In [16]:
# space-removal bandaids that are now blocked with an "empty" response.
bandaids = hate_delta[
    (hate_delta.status_no_spaces_after != hate_delta.status_no_spaces_before) & 
    (~hate_delta.status_no_spaces_before.isnull())
][['search_term','status_no_spaces_before', 'status_no_spaces_after']]
bandaids.search_term = bandaids.search_term.str.replace(' ', '')
bandaids.set_index('search_term')

,status_no_spaces_before,status_no_spaces_after
search_term,,
dualseedline,Full,Empty
globalistjews,Full,Empty
heilhitler,Full,Empty
jewishquestion,Full,Empty
radicalislamicterror,Full,Empty
siegheil,Full,Empty
whitegenocide,Full,Empty
whitenationalist,Full,Empty
whitepride,Full,Empty


Let's display these changed and unchanged terms in a side-by-side table:

In [17]:
def what_is_still_available(df) -> list:
    """
    Filters unaltered and space-removed terms that remain "Full"
    """
    # which of the space-removal bandaid terms have not changed?
    still_available  = df[
        (df.status_no_spaces_after == df.status_no_spaces_before) & 
        (df.status_no_spaces_before == 'Full')
    ].search_term.tolist()
    still_available  = [w.replace(' ', '') for w in still_available]

    # which unaltered terms have not changed status?
    still_available += df[
        (df.status_after == 'Full')
    ].search_term.tolist()

    return sorted(still_available)

def what_changed(df) -> list:
    """
    Filtered unaltered and space-removed terms that changed.
    We assume that change is from "Full" to "Empty".
    """
    # which of the space-removal bandaid terms have changed?
    newly_blocked = df[
        (df.status_no_spaces_after != df.status_no_spaces_before) & 
        (~df.status_no_spaces_before.isnull())
    ].search_term.tolist()
    newly_blocked = [w.replace(' ', '') for w in newly_blocked]
    
    # which unaltered terms have changed status?
    newly_blocked += df[
        df.status_after != df.status_before
    ].search_term.tolist()

    return sorted(newly_blocked)

def where_are_we_now(df) -> pd.DataFrame:
    """
    Displays what is still "Full" response with those that changed.
    In our data, all changed terms went from "Full" to "Empty".
    These new "Empty" responses are wholly unique from our reporting.
    """
    still_available = what_is_still_available(df)
    newly_blocked = what_changed(df)
    
    # make the arrays the same length
    n_pad = abs(len(newly_blocked) - len(still_available))
    if len(still_available) < len(newly_blocked):
        still_available.extend([None for _ in range(n_pad)])
    elif len(still_available) > len(newly_blocked):
        newly_blocked.extend([None for _ in range(n_pad)])
        
    changed_terms =pd.DataFrame({
        'still allowed for placements' : still_available, 
        'now blocked from placements': newly_blocked
    })
    
    return changed_terms

In [18]:
changed_hate_terms = where_are_we_now(hate_delta)
changed_hate_terms 

,still allowed for placements,now blocked from placements
0,american renaissance,14 words
1,amerimutt,2083: a european declaration of independence
2,black sun,alt-lite
3,boogaloo,alt-right
4,color of crime,american front
5,diversity is a code word for anti-white,american identity movement
6,hammerskin,american vanguard
7,national socialism,blood and soil
8,open borders for israel,civilization jihad
9,race and iq,council of conservative citizens


In [19]:
changed_hate_terms.to_csv(fn_hate_table4, index=False)

In [20]:
# "terrorists" vs "terrorist"
(word[word.search_term.str.contains('terroris')]
    .append(policy[policy.search_term.str.contains('terroris')])
    .append(adhoc[adhoc.search_term.str.contains('terroris')])
    .sort_values(by='status')[display_cols]
    .set_index('search_term'))

,status,status_no_spaces
search_term,,
praise terrorist,Blocked,NaN
terrorist ideology,Blocked,Empty
terrorist hostages,Blocked,Empty
terrorist acts,Blocked,Empty
terrorist recruitment,Blocked,NaN
terrorist attack,Blocked,NaN
terrorism,Blocked,NaN
terrorism acts,Blocked,NaN
recruit terrorist,Blocked,NaN


## Analyze Social Justice terms

In [21]:
social_justice_og = pd.read_csv(fn_social_justice_og)

In [22]:
# before
value_counts(social_justice_og, col='status')

,count,percentage
Full,42,0.677419
Blocked,17,0.274194
Partial Block,3,0.048387


In [23]:
# after: half the terms are now blocked with an "empty" response
value_counts(social_justice, col='status')

,count,percentage
Empty,32,0.516129
Blocked,17,0.274194
Full,10,0.161290
Partial Block,3,0.048387


In [24]:
social_justice_delta = social_justice_og[display_cols].merge(
    social_justice[display_cols],
    on= 'search_term', 
    suffixes= ['_before', '_after']
)

In [25]:
# Which terms got blocked?
social_justice_delta[
    social_justice_delta.status_before != social_justice_delta.status_after
]

,search_term,status_before,status_no_spaces_before,status_after,status_no_spaces_after
0,abolish ice,Full,NaN,Empty,NaN
2,anti-black,Full,NaN,Empty,NaN
3,anti-fascist,Full,NaN,Empty,NaN
4,antiracism,Full,NaN,Empty,NaN
5,believe black women,Full,NaN,Empty,NaN
6,bipoc,Full,NaN,Empty,NaN
7,black august,Full,NaN,Empty,NaN
9,black excellence,Full,NaN,Empty,NaN
10,black girls matter,Full,NaN,Empty,NaN
12,black identity extremists,Full,NaN,Empty,NaN


In [26]:
changed_socjust_term = where_are_we_now(social_justice_delta)
changed_socjust_term

,still allowed for placements,now blocked from placements
0,abolish the police,abolish ice
1,black dissent,americanmuslim
2,black hair,anti-black
3,digital justice,anti-fascist
4,for the culture,antiracism
5,hijab fashion,believe black women
6,no tech for ice,bipoc
7,oscars so white,black august
8,patriarchy,black excellence
9,tell black stories,black girls matter


In [27]:
changed_socjust_term.to_csv(fn_socjust_table4, index=False)

## How did Google treat keywords related to religions?
For each topic, we collected "`{religion} {topic}`". Where `religion` was "mulsim," "christian," "jewish," and "buddhist".

In [28]:
topics = [
    'culture', 
    'diet', 
    'holidays', 
    'history', 
    'conversion', 
    'parenting', 
    'fashion'
]

In [29]:
# "Blocked" responses were originally blocked, "Empty" responses are newly blocked.
for base in topics:
    print(f">>> X {base} >>>")
    files_religion = glob.glob(f'{data_dir_religions}*{base}.json')
    for fn in sorted(files_religion, reverse=True):
        resp = process_api_response(fn)
        status = determine_status(resp)
        term = fn.split('/')[-1].replace('.json', '')
        print(f"{term}: {status}")
    print()

>>> X culture >>>
muslim culture: Blocked
jewish culture: Empty
christian culture: Empty
buddhist culture: Empty

>>> X diet >>>
muslim diet: Blocked
jewish diet: Blocked
christian diet: Blocked
buddhist diet: Empty

>>> X holidays >>>
muslim holidays: Blocked
jewish holidays: Empty
christian holidays: Empty
buddhist holidays: Empty

>>> X history >>>
muslim history: Blocked
jewish history: Empty
christian history: Empty
buddhist history: Empty

>>> X conversion >>>
muslim conversion: Blocked
jewish conversion: Blocked
christian conversion: Blocked
buddhist conversion: Blocked

>>> X parenting >>>
muslim parenting: Blocked
jewish parenting: Blocked
christian parenting: Empty
buddhist parenting: Blocked

>>> X fashion >>>
muslim fashion: Blocked
jewish fashion: Empty
christian fashion: Empty
buddhist fashion: Empty

